# Creating simple web app via Flask and Streamlit

1) In command line: python server_ml.py
2) In another command line: streamlit run app.py

### Creating Streamlit web page

In [1]:
%%writefile app.py

import streamlit as st
import numpy as np
from cv2 import imread
import requests
import json
import base64


# All interactions with user
def interactions():
    title_text = st.sidebar.selectbox("Which title has your Passenger?",
                                     ('Master', 'Young Miss', 'Miss', 'Mr', 'Mrs', 'Noble', 'Other'))
    
    p_class = st.sidebar.radio("Social-Economical Passenger Status: ",
                                   ("Upper Class", "Middle Class", 'Lower Class'),
                                   key='P_class')
    
    fare_num = st.sidebar.number_input("Ticket cost per family member: ", 0.0, 600.0, step=5.0, key='Fare_num')
    
    sex = st.sidebar.radio("Sex: ", ("Female", "Male"), key='Sex')
    
    age_num = st.sidebar.slider("Age of Passenger: ", 1, 80, key='Age_num')

    family_num = st.sidebar.slider("How many family members on the bort?", 0, 10, key='Family_num')
    
    if family_num != 0:
        family_surv = st.sidebar.radio("Anybody survived in this family?", ('Yes', "No"), key='Family_surv')
    else:
        family_surv = 'Unknown'
        
    return p_class, sex, age_num, family_num, family_surv, fare_num, title_text
        

#Read image from file    
@st.cache(allow_output_mutation=True)
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()


#Set backgroud image
def set_png_as_page_bg(png_file):
    bin_str = get_base64_of_bin_file(png_file)
    page_bg_img = '''
    <style>
    body {
    background-image: url("data:image/jpg;base64,%s");
    background-size: cover;
    }
    </style>
    ''' % bin_str
    
    st.markdown(page_bg_img, unsafe_allow_html=True)
    return


# Set text style
def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)
        return


def main():
    URI = 'http://127.0.0.1:5000'
    
    local_css('text.css')
    set_png_as_page_bg('background.jpg')
    
    st.title("Titanic disaster")
    st.header('The wreck of Titanic - a sea disaster that occurred 1912 in the North Atlantic Ocean')
    st.markdown("Can you create a passenger that could survive in this terrible disaster?")
    st.image(imread('titanic.jpg'), width = 400)
    
    st.sidebar.markdown("""<font color="black"
    font-family = "Times"
    font-size = "14pt"
    font-style = "normal"
    font-weight = "Normal">Let's select the characteristics of your Passenger</font>""", unsafe_allow_html=True)

    p_class, sex, age_num, family_num, family_surv, fare_num, title_text = interactions()
    
    if st.sidebar.button('Get predictions'):
        # Send data to server in 'data'
        response = requests.post(URI, data={'p_class':p_class,
                                           'sex':sex,
                                           'age_num':age_num,
                                           'family_num':family_num,
                                           'family_surv':family_surv,
                                           'fare_num':fare_num,
                                           'title_text':title_text})
        #read json response
        response = json.loads(response.text)
        #take information
        preds = response.get('prediction')[0][0]
                    
        st.markdown(f"The Passenger's probability of surviving is {round(preds*100, 2)}%")
    return

if __name__ == '__main__':
    main()

Overwriting app.py


### Creating server for data preprocessing and Keras model

In [2]:
%%writefile server_ml.py

import json
import tensorflow as tf
import numpy as np
import os
import random
import string
from flask import Flask, request

app = Flask(__name__)

# Preprocessing data like in prepering data to modeling
def pclass_preprocessing(p_class):
    dict_pclass = {'Upper Class':0, 'Middle Class':1, 'Lower Class':2}
    choosen_indx= dict_pclass[p_class]
    
    pclass = np.zeros((1,len(dict_pclass)))
    pclass[0,choosen_indx] = 1
    return pclass

def sex_preprocessing(sex):
    dict_sex = {'Female': 0, 'Male': 1}
    choosen_indx = dict_sex[sex]
    
    sex = np.zeros((1,len(dict_sex)))
    sex[0, choosen_indx] = 1
    return sex


def age_preprocessing(age_num):
    age_group = np.zeros((1,5))
    if 0 < age_num < 15:
        age_group[0, 0] = 1
    elif 15 <= age_num < 30:
        age_group[0, 1] = 1
    elif 30 <= age_num < 45:
        age_group[0, 2] = 1
    elif 45 <= age_num < 60:
        age_group[0, 3] = 1
    else:
        age_group[0, 4] = 1
    return age_group


def family_preprocessing(family_num):
    family_size = np.zeros((1,3))
    if family_num == 0:
        family_size[0,1] = 1
    elif 0 < family_num <= 3:
        family_size[0,2] = 1
    else:
        family_size[0,0] = 1
    return family_size



def family_surv_preprocessing(family_surv):
    dict_survived = {'Yes':2, 'No':0, 'Unknown':1}
    choosen_indx = dict_survived[family_surv]
    
    family_survived = np.zeros((1,len(dict_survived)))
    family_survived[0,choosen_indx] = 1
    return family_survived


def fare_preprocessing(fare_num):
    fare_group = np.zeros((1,7))                                                                                       
    if -0.001< fare_num <= 7.0:
        fare_group[0,0]=1
    elif 7.0< fare_num <= 7.75:
        fare_group[0,1]=1
    elif 7.75< fare_num <= 7.925:
        fare_group[0,2]=1                                                                                             
    elif 7.925< fare_num <= 9.588:
        fare_group[0,3]=1
    elif 9.588< fare_num <= 13.0:
        fare_group[0,4]=1
    elif 13.0< fare_num <= 27.721:
        fare_group[0,5]=1
    else:
        fare_group[0,6]=1
    return fare_group
    
        
def title_preprocessing(title_text):
    dict_title = {'Master':0, 'Young Miss':6, 'Miss':1, 'Mr':2, 'Mrs':3, 'Noble':4, 'Other':5}
    choosen_indx = dict_title[title_text]
    
    title = np.zeros((1,len(dict_title)))
    title[0,choosen_indx] = 1
    return title


# Stack all arrays together into one arrray for neural network
def preprocessing_all_data(p_class, sex, age_num, family_num, family_surv, fare_num, title_text):
    pclass = pclass_preprocessing(p_class)
    sex = sex_preprocessing(sex)
    age_group = age_preprocessing(age_num)
    family_size = family_preprocessing(family_num)
    family_survived = family_surv_preprocessing(family_surv)
    fare_group = fare_preprocessing(fare_num)
    title = title_preprocessing(title_text)
    result = np.hstack((pclass, sex, title, age_group, family_size, family_survived, fare_group))
    
    return result


#load our base model
model = tf.keras.models.load_model('basemodel.hdf5')


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        
        # Retrieve information from POST request
        p_class = request.form.get('p_class')
        sex = request.form.get('sex')
        age_num = int(request.form.get('age_num'))
        family_num= int(request.form.get('family_num'))
        family_surv= request.form.get('family_surv')
        fare_num = float(request.form.get('fare_num'))
        title_text = request.form.get('title_text')
        
        # preprocessing data
        result = preprocessing_all_data(p_class, sex, age_num, family_num, family_surv, fare_num, title_text)
        
        #Make predict
        predictions = model.predict(result)
        
        #Return in streamlit
        return json.dumps({'prediction':predictions.tolist()})
    
    return 'Welcome to the ml server'



if __name__ == '__main__':
    app.run(debug=True)

Overwriting server_ml.py


#### Auxilary css file for streamlit

In [3]:
%%writefile text.css

/*Header text*/
h1 {
    font-family: Times;
    font-style : normal;
    font-weight: Normal;
    font-size: 14pt;
    text-align: center;
    color: #ffffff;
}

/* markdown text */
body {
    font-family: Georgia;
    font-style : normal;
    font-weight: Normal;
    font-size: 12pt;
    text-align: center;
    color: #ffffff;
}

Overwriting text.css
